In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
melb = pd.read_csv('../../data/melb_data.csv')

In [3]:
# three approaches
# 1) drop columns with missing values
# 2) imputation
# 3) an extension to imputation

In [3]:
# 1 approach
y = melb.Price
melb_predictors = melb.drop(['Price'], axis=1)

In [4]:
X = melb_predictors.select_dtypes(exclude=['object'])

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [6]:
X_train

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.98670,13240.0
6524,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.90050,6380.0
8413,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.82200,3755.0
2919,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.91580,8870.0
6043,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13123,3,5.2,3056.0,3.0,1.0,2.0,212.0,NaN,NaN,-37.77695,144.95785,11918.0
3264,3,10.5,3081.0,3.0,1.0,1.0,748.0,101.0,1950.0,-37.74160,145.04810,2947.0
9845,4,6.7,3058.0,4.0,2.0,2.0,441.0,255.0,2002.0,-37.73572,144.97256,11204.0
10799,3,12.0,3073.0,3.0,1.0,1.0,606.0,NaN,NaN,-37.72057,145.02615,21650.0


In [7]:
X_val

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
8505,4,8.0,3016.0,4.0,2.0,2.0,450.0,190.0,1910.0,-37.86100,144.89850,6380.0
5523,2,6.6,3011.0,2.0,1.0,0.0,172.0,81.0,1900.0,-37.81000,144.88960,2417.0
12852,3,10.5,3020.0,3.0,1.0,1.0,581.0,NaN,NaN,-37.76740,144.82421,4217.0
4818,3,4.5,3181.0,2.0,2.0,1.0,128.0,134.0,2000.0,-37.85260,145.00710,7717.0
12812,3,8.5,3044.0,3.0,2.0,2.0,480.0,NaN,NaN,-37.72523,144.94567,7485.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2664,2,6.4,3011.0,2.0,1.0,1.0,47.0,35.0,2013.0,-37.80140,144.89590,7570.0
8513,4,8.0,3016.0,4.0,2.0,4.0,551.0,NaN,NaN,-37.85790,144.87860,6380.0
12922,3,10.8,3105.0,3.0,1.0,1.0,757.0,NaN,NaN,-37.78094,145.10131,4480.0
10761,4,6.2,3039.0,4.0,1.0,3.0,478.0,152.0,1925.0,-37.76421,144.90571,6232.0


In [8]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

In [9]:
cols_with_missing

['Car', 'BuildingArea', 'YearBuilt']

In [12]:
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_val = X_val.drop(cols_with_missing, axis=1)

print('MAE from Approach 1 (Drop columns with missing values):')
print(score_dataset(reduced_X_train, reduced_X_val, y_train, y_val))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


In [14]:
# 2 approach
from sklearn.impute import SimpleImputer

In [13]:


my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(my_imputer.transform(X_val))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_val, y_train, y_val))

MAE from Approach 2 (Imputation):


NameError: name 'score_dataset' is not defined

In [10]:
# 3 approach



# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_val_plus = X_val.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_val_plus[col + '_was_missing'] = X_val_plus[col].isnull()





In [11]:
X_train_plus

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_was_missing,BuildingArea_was_missing,YearBuilt_was_missing
12167,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.98670,13240.0,False,True,False
6524,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.90050,6380.0,False,True,True
8413,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.82200,3755.0,False,True,True
2919,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.91580,8870.0,False,True,False
6043,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.82720,4217.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,3,5.2,3056.0,3.0,1.0,2.0,212.0,NaN,NaN,-37.77695,144.95785,11918.0,False,True,True
3264,3,10.5,3081.0,3.0,1.0,1.0,748.0,101.0,1950.0,-37.74160,145.04810,2947.0,False,False,False
9845,4,6.7,3058.0,4.0,2.0,2.0,441.0,255.0,2002.0,-37.73572,144.97256,11204.0,False,False,False
10799,3,12.0,3073.0,3.0,1.0,1.0,606.0,NaN,NaN,-37.72057,145.02615,21650.0,False,True,True


In [15]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_val_plus = pd.DataFrame(my_imputer.transform(X_val_plus))



In [18]:
imputed_X_train_plus

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_was_missing,BuildingArea_was_missing,YearBuilt_was_missing
0,1.0,5.0,3182.0,1.0,1.0,1.0,0.0,153.764119,1940.000000,-37.85984,144.98670,13240.0,0.0,1.0,0.0
1,2.0,8.0,3016.0,2.0,2.0,1.0,193.0,153.764119,1964.839866,-37.85800,144.90050,6380.0,0.0,1.0,1.0
2,3.0,12.6,3020.0,3.0,1.0,1.0,555.0,153.764119,1964.839866,-37.79880,144.82200,3755.0,0.0,1.0,1.0
3,3.0,13.0,3046.0,3.0,1.0,1.0,265.0,153.764119,1995.000000,-37.70830,144.91580,8870.0,0.0,1.0,0.0
4,3.0,13.3,3020.0,3.0,1.0,2.0,673.0,673.000000,1970.000000,-37.76230,144.82720,4217.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,3.0,5.2,3056.0,3.0,1.0,2.0,212.0,153.764119,1964.839866,-37.77695,144.95785,11918.0,0.0,1.0,1.0
10860,3.0,10.5,3081.0,3.0,1.0,1.0,748.0,101.000000,1950.000000,-37.74160,145.04810,2947.0,0.0,0.0,0.0
10861,4.0,6.7,3058.0,4.0,2.0,2.0,441.0,255.000000,2002.000000,-37.73572,144.97256,11204.0,0.0,0.0,0.0
10862,3.0,12.0,3073.0,3.0,1.0,1.0,606.0,153.764119,1964.839866,-37.72057,145.02615,21650.0,0.0,1.0,1.0


In [17]:
# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_val_plus.columns = X_val_plus.columns

In [21]:
from sklearn.impute import MissingIndicator
import numpy as np

In [83]:
X_1 = np.array([[np.nan, np.nan, 3],
                [np.nan, 4, 4],
                [1, 9, 3]])
X_2 = np.array([[np.nan, 6, 1],
                [7, 8, 5],
                [np.nan, 11, 5]])

In [84]:
X_1 = pd.DataFrame(X_1)
X_2 = pd.DataFrame(X_2)

In [85]:
X_1

,0,1,2
0,NaN,NaN,3.0
1,NaN,4.0,4.0
2,1.0,9.0,3.0


In [86]:
X_2

,0,1,2
0,NaN,6.0,1.0
1,7.0,8.0,5.0
2,NaN,11.0,5.0


In [87]:
indi = MissingIndicator()

In [91]:
indi.fit_transform(X_1)

array([[ True,  True],
       [ True, False],
       [False, False]])

In [89]:
X_3 = indi.transform(X_2)

In [90]:
X_3

array([[ True, False],
       [False, False],
       [ True, False]])

In [ ]:


print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_val_plus, y_train, y_val))

In [15]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


In [17]:
df_train = pd.read_csv('../../data/home-data-for-ml-course/train.csv', index_col='Id')

df_test = pd.read_csv('../../data/home-data-for-ml-course/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
df_train.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = df_train.SalePrice
df_train.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = df_train.select_dtypes(exclude=['object'])
X_test = df_test.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)


In [18]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [19]:
# Function for comparing different approaches
def score_data(X_train, X_val, y_train, y_val):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return mean_absolute_error(y_val, preds)

In [20]:
# Fill in the line below: get names of columns with missing values
miss = [col for col in X_train.columns if X_train[col].isnull().any()] # Your code here

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(miss, axis=1)
reduced_X_val = X_val.drop(miss, axis=1)

In [21]:
print("MAE (Drop columns with missing values):")
print(score_data(reduced_X_train, reduced_X_val, y_train, y_val))

MAE (Drop columns with missing values):
17837.82570776256


In [22]:
# Fill in the lines below: imputation
imputer = SimpleImputer() # Your code here
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(imputer.transform(X_val))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns


In [23]:
print("MAE (Imputation):")
print(score_data(imputed_X_train, imputed_X_val, y_train, y_val))

MAE (Imputation):
18062.894611872147


In [24]:
final_X_train = X_train.drop(miss, axis=1)
final_X_val = X_val.drop(miss, axis=1)

In [25]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_val = model.predict(final_X_val)
print("MAE (Your approach):")
print(mean_absolute_error(y_val, preds_val))

MAE (Your approach):
17837.82570776256


In [26]:
# Fill in the line below: preprocess test data
X_test = X_test.drop(miss, axis=1)
my_imputer = SimpleImputer()
imputed_X_test = pd.DataFrame(my_imputer.fit_transform(X_test))


# Imputation removed column names; put them back
imputed_X_test.columns = X_test.columns


final_X_test = imputed_X_test

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)

In [27]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission_2.csv', index=False)